In [1]:
import os
NOTEBOOK_NAME = "ex8-finetune-constractive-loss-ep6-10"
os.environ['WANDB_NOTEBOOK_NAME'] = NOTEBOOK_NAME

In [2]:
OUTPUT_DIR = f"/notebooks/kaggle_lecr/output/{NOTEBOOK_NAME}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
!nvidia-smi

Fri Feb 24 12:27:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:00:05.0 Off |                  Off |
| 30%   38C    P8    26W / 300W |   3274MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
os.system('pip install python-dotenv')

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
class CFG:
    debug = False
    upload_data = True
    wandb = True
    batch_size = 128# 48GBならbs128, 24GBならbs64でいける, 16GBならbs32
    n_folds = 5
    seed = 42
    num_epochs = 5#5#3#10
    data_url = "/notebooks/kaggle_lecr/data/learning-equality-curriculum-recommendations/"
    # uns_model = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    uns_model = "/notebooks/kaggle_lecr/output/ex8-finetune-constractive-loss-ep1-5"

In [6]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    os.system('pip install wandb')
    import wandb

    try:
        # for kaggle
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        
        # for paperspace
        secret_value_0 = os.getenv('WANDB_API_KEY')
        wandb.login(key=secret_value_0)
        
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='LECR',
                     entity="sinchir0",
                     name=NOTEBOOK_NAME,
                     config=class2dict(CFG),
                     group="fine-tuning",
                     job_type="train",
                     anonymous=anony)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ex8-finetune-constractive-loss-ep6-10.
wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
os.system("pip3 install torch==1.12.0 --extra-index-url https://download.pytorch.org/whl/cu116")

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.12.0%2Bcu116-cp39-cp39-linux_x86_64.whl (1904.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 1.12.0+cu116 which is incompatible.


0

In [8]:
import torch
if CFG.wandb:
    wandb.log({"GPU":torch.cuda.get_device_name()})

In [9]:
os.system("pip install scikit-learn==1.2.1")

0

In [10]:
!pip -qqq install sentence-transformers
!pip -qqq install datasets
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from datasets import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedGroupKFold, KFold, StratifiedKFold, GroupKFold

In [11]:
DATA_PATH = "/kaggle/input/learning-equality-curriculum-recommendations/"
topics = pd.read_csv(CFG.data_url + "topics.csv")
content = pd.read_csv(CFG.data_url + "content.csv")
correlations = pd.read_csv(CFG.data_url + "correlations.csv")
correlations.shape

(61517, 2)

In [12]:
topics = topics.fillna("")
content = content.fillna("")

In [13]:
topics

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
76969,t_fffe14f1be1e,Lección 7,,6e90a7,aligned,6,es,t_d448c707984d,True
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...,5b9e5ca86571f90499ea987f,9fd860,source,2,ar,t_5b4f3ba4eb7d,True


In [14]:
def add_parent_text(topics: pd.DataFrame):
    id_full_text = {}
    id_to_text = {}
    for i, row in topics.iterrows():
        id_to_text[row.id] = [row.title, row.parent]
    print('done')
    def get_full_text(id):
        if id in id_full_text:
            return id_full_text[id]
        data = id_to_text[id]
        # full_text = f'{data[0]} < {get_full_text(data[1])}' if data[1] != '' else data[0]
        full_text = f'{data[0]} > {get_full_text(data[1])}' if data[1] != '' else data[0]
        id_full_text[id] = full_text
        return full_text
    tqdm.pandas()
    topics['title'] = topics.id.progress_apply(get_full_text)
    del id_full_text
    del id_to_text
    return topics

topics = add_parent_text(topics)

done


  0%|          | 0/76972 [00:00<?, ?it/s]

In [15]:
train = pd.read_csv("/notebooks/kaggle_lecr/data/lecr-uns-top-n-10-para-mpnet-data/train.csv")

In [16]:
train = train.rename(columns={"topics_titles":"topics_texts", "content_titles":"content_texts"})

In [17]:
# positive sampleを足す
def add_positive_sample(train: pd.DataFrame, correlations: pd.DataFrame, topics: pd.DataFrame, content: pd.DataFrame):
    
    # topic_text_dict = dict(zip(topics["id"], topics['title'] + " " + topics['description']))
    # content_text_dict = dict(zip(content["id"], content['title'] + " " + content['description']))
    topic_text_dict = dict(zip(topics["id"], topics['title']))
    content_text_dict = dict(zip(content["id"], content['title']))
    
    correlations["content_ids_list"] = correlations["content_ids"].apply(lambda x : x.split())
    
    all_positive_sample = correlations.explode("content_ids_list")[["topic_id","content_ids_list"]]
    all_positive_sample = all_positive_sample.rename(columns={"topic_id":"topics_ids","content_ids_list":"content_ids"})
    
    all_positive_sample["topics_texts"] = all_positive_sample["topics_ids"].map(topic_text_dict)
    all_positive_sample["content_texts"] = all_positive_sample["content_ids"].map(content_text_dict)
    all_positive_sample["target"] = 1
    
    all_positive_sample = all_positive_sample.reset_index(drop=True)
    
    # 追加するtopic, contentのみを持つlistを生成
    all_positive_topic_content = (all_positive_sample["topics_ids"] + all_positive_sample["content_ids"]).tolist()
    train_positive = train[train["target"] == 1]
    train_positive_topic_content = (train_positive["topics_ids"] + train_positive["content_ids"]).tolist()
    add_topic_content = list(set(all_positive_topic_content) - set(train_positive_topic_content))
    
    # trainにpositive sampleを追加
    train = pd.concat([train, all_positive_sample]).drop_duplicates(subset=["topics_ids","content_ids"], keep='first')
    train = train.sort_values("topics_ids")
    train = train.reset_index(drop=True)

    return train, add_topic_content

train, add_topic_content = add_positive_sample(train, correlations, topics, content)

In [18]:
train.head()

,topics_ids,content_ids,topics_texts,content_texts,target
0,t_00004da3a1b2,c_dfa229bd21df,Откриването на резисторите > Открития и проект...,Последователно свързани резистори,0
1,t_00004da3a1b2,c_376c5a8eb028,Откриването на резисторите > Открития и проект...,Да чуем променливото съпротивление,1
2,t_00004da3a1b2,c_8e6555906046,Откриването на резисторите > Открития и проект...,Пример: Анализиране на по-сложна верига от рез...,0
3,t_00004da3a1b2,c_0b4a3ea959ba,Откриването на резисторите > Открития и проект...,Последователни и успоредни резистори,0
4,t_00004da3a1b2,c_e1e8557d7c61,Откриването на резисторите > Открития и проект...,Последователни резистори,0


In [19]:
train_samples = []
for _, row in tqdm(train.iterrows(), total=train.shape[0]):
    sample = InputExample(texts=[row['topics_texts'], row['content_texts']], label=row['target'])
    train_samples.append(sample)

  0%|          | 0/734999 [00:00<?, ?it/s]

In [20]:
model = SentenceTransformer(CFG.uns_model)

In [21]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.5
margin = 0.5

train_dataloader = DataLoader(
    train_samples, 
    shuffle=True, 
    batch_size=CFG.batch_size,
    num_workers=os.cpu_count(),
    pin_memory=True
)
train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [22]:
if CFG.debug:
    CFG.num_epochs = 1
warmup_steps = int(len(train_dataloader) * CFG.num_epochs * 0.1) #10% of train data

In [ ]:
# TODO: 公式の使い方見て良さげな関数入れるAWPとか
# https://www.sbert.net/docs/training/overview.html
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=CFG.num_epochs,
          save_best_model = True,
          output_path=OUTPUT_DIR,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5743 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5743 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5743 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5743 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5743 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Upload

In [ ]:
import os
os.system("pip install kaggle")
os.system("mkdir -p ~/.kaggle/")
os.system("cp /notebooks/kaggle_lecr/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
import json

def dataset_create_new(dataset_name: str, upload_dir: str):
    if "_" in dataset_name:
        raise ValueError("datasetの名称に_の使用は禁止です")
    dataset_metadata = {}
    dataset_metadata['id'] = f'sinchir0/{dataset_name}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = dataset_name
    with open(os.path.join(upload_dir, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode='tar')

if CFG.upload_data:
    print(f"Create Dataset name:{NOTEBOOK_NAME}, output_dir:{OUTPUT_DIR}")
    dataset_create_new(dataset_name=NOTEBOOK_NAME, upload_dir=OUTPUT_DIR)

In [ ]:
wandb.finish()